In [2]:
import sys, os
from nn.wandb_classification_callback import WandbClassificationCallback


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # tensorflow を読み込む前のタイミングですると効果あり
import tensorflow as tf

tf.random.set_seed(100)
from data_analysis.utils import Utils
import sys
import datetime
import wandb
from wandb.keras import WandbCallback
from pre_process.pre_process import PreProcess
from nn.model_base import EDLModelBase, edl_classifier_1d
from nn.losses import EDLLoss
from pre_process.json_base import JsonBase
from data_analysis.py_color import PyColor
from collections import Counter
from nn.utils import load_model, separate_unc_data
from mywandb.utils import make_ss_dict4wandb
# 環境設定
CALC_DEVICE = "gpu"
# CALC_DEVICE = "cpu"
DEVICE_ID = "0" if CALC_DEVICE == "gpu" else "-1"
os.environ["CUDA_VISIBLE_DEVICES"] = DEVICE_ID
if os.environ["CUDA_VISIBLE_DEVICES"] != "-1":
    tf.keras.backend.set_floatx("float32")
    physical_devices = tf.config.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    # tf.config.run_functions_eagerly(True)
else:
    print("*** cpuで計算します ***")
    # なんか下のやつ使えなくなっている、、
    # tf.config.run_functions_eagerly(True)

In [14]:
TEST_RUN = False
HAS_ATTENTION = True
PSE_DATA = False
HAS_INCEPTION = True
IS_PREVIOUS = False
IS_NORMAL = True
IS_ENN = False
IS_MUL_LAYER = False
CATCH_NREM2 = True
HAS_NR2_BIAS = True
EPOCHS = 10
BATCH_SIZE = 512
N_CLASS = 5
KERNEL_SIZE = 128
STRIDE = 16
SAMPLE_SIZE = 10000
UNC_THRETHOLD = 0.2
DATA_TYPE = "spectrogram"
FIT_POS = "middle"
EXPERIMENT_TYPES = (
    "no_cleansing",
    "positive_cleansing",
    "negative_cleansing",
)
EXPERIENT_TYPE = "no_cleansing"
NORMAL_TAG = "normal" if IS_NORMAL else "sas"
ATTENTION_TAG = "attention" if HAS_ATTENTION else "no-attention"
PSE_DATA_TAG = "psedata" if PSE_DATA else "sleepdata"
INCEPTION_TAG = "inception" if HAS_INCEPTION else "no-inception"
WANDB_PROJECT = "test" if TEST_RUN else "enn4fixed_stride_fixed_sample"
ENN_TAG = "enn" if IS_ENN else "dnn"
INCEPTION_TAG += "v2" if IS_MUL_LAYER else ""
CATCH_NREM2_TAG = "catch_nrem2" if CATCH_NREM2 else "catch_nrem34"

In [15]:
pre_process = PreProcess(
    data_type=DATA_TYPE,
    fit_pos=FIT_POS,
    verbose=0,
    kernel_size=KERNEL_SIZE,
    is_previous=IS_PREVIOUS,
    stride=STRIDE,
    is_normal=IS_NORMAL,
    has_nrem2_bias=True,
    has_rem_bias=False,
    model_type=ENN_TAG,
    cleansing_type="no_cleansing",
    make_valdata=True,
    has_ignored=True,
    lsp_option="nr2",
)

In [16]:
MI = pre_process.my_env.mi
datasets = pre_process.load_sleep_data.load_data(
    load_all=True,
    pse_data=PSE_DATA,
)

*** すべての被験者を読み込みます（load_dataの引数:nameは無視します） ***
 140703_Li を読み込みます 
 140711_Yamamoto を読み込みます 
 140821_Murakami を読み込みます 
 140823_Murakami を読み込みます 
 140826_Ootsuji を読み込みます 
 140828_Otsuji を読み込みます 
 140830_Murakami を読み込みます 
 140929_Kumazawa を読み込みます 
 140930_Hayashi を読み込みます 
 141002_Hayashi を読み込みます 
 141006_Hiromoto を読み込みます 
 141015_Hiromoto を読み込みます 
 141024_Kawasaki_F を読み込みます 
 141027_Kawasaki を読み込みます 
 141029_Umezawa を読み込みます 
 141104_Takadama を読み込みます 
 141124_Murata を読み込みます 
 141127_Tatsumi を読み込みます 
 141128_Sato_Minato を読み込みます 
 141204_Tatebe を読み込みます 
 141205_Fujitsuka を読み込みます 
 141217_Sugimoto を読み込みます 
 141219_Usui を読み込みます 
 151104_Nagae を読み込みます 
 151105_Nagae を読み込みます 
 151106_Nagae を読み込みます 
 151111_Kawasaki を読み込みます 
 151112_Kawasaki を読み込みます 
 151113_Kawasaki を読み込みます 
 151118_Tomoko_Nagae を読み込みます 
 151119_Tomoko_Nagae を読み込みます 
 151126_Tanaka を読み込みます 
 151127_Tanaka を読み込みます 
 151201_Tomoko_Nagae を読み込みます 
 151204_Umenai を読み込みます 
 151207_Matsumoto を読み込みます 
 151214_Matsumoto を読み込みます 
 15121

In [20]:
test_id = 0
test_name = pre_process.name_list[test_id]
(train, test) = pre_process.split_train_test_from_records(datasets, test_id=test_id, pse_data=False)
utils=Utils(IS_NORMAL, IS_PREVIOUS, DATA_TYPE, FIT_POS,
            STRIDE, KERNEL_SIZE, model_type=ENN_TAG,
            cleansing_type="no_cleansing")

In [ ]:
# モデルの読み込み
loaded_name = "140703_Li"
date_id = {"nothing":"20211010-150603", "negative":"20211010-150603", "positive":"20211010-150603"}
n_class = 5
model = load_model(loaded_name=loaded_name, model_id=date_id, n_class=n_class, verbose=0)
# model.summary()